In [824]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
%matplotlib inline
import numpy as np
import pandas as pd
import datetime
import random
import warnings
warnings.filterwarnings('ignore')

# Plots
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import pandas_profiling 
from mlxtend.plotting import plot_learning_curves

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from math import *
from sklearn import metrics

# Misc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [0]:
train = pd.read_csv("/content/drive/My Drive/allstate-claims-severity/train.csv")

In [0]:
test = pd.read_csv("/content/drive/My Drive/allstate-claims-severity/test.csv")

In [0]:
submission_file = pd.read_csv("/content/drive/My Drive/allstate-claims-severity/sample_submission.csv")

In [828]:
train.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,...,A,D,D,C,E,E,A,D,L,O,A,B,E,F,H,F,A,AB,DK,A,C,AF,A,I,GK,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,C,E,E,D,T,I,D,A,A,E,E,I,K,K,BI,CS,C,N,AE,A,O,DJ,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,H,D,B,D,E,E,A,P,F,J,A,A,D,E,K,G,B,H,C,C,Y,BM,A,K,CK,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [829]:
test.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,A,D,C,C,E,C,D,T,H,G,A,A,G,E,I,L,K,BI,BC,A,J,AX,A,Q,HG,0.321594,0.299102,0.246911,0.402922,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,A,D,D,D,E,A,A,P,B,D,A,A,G,G,G,F,B,BI,CO,E,G,X,A,L,HK,0.634734,0.620805,0.654310,0.946616,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,B,A,B,B,B,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,B,A,B,B,...,A,A,D,D,C,E,E,A,D,G,Q,A,D,D,E,J,G,A,BI,CS,C,U,AE,A,K,CK,0.290813,0.737068,0.711159,0.412789,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,B,A,...,G,H,D,D,C,E,E,D,T,G,A,A,D,E,E,I,K,K,BI,CR,A,AY,AJ,A,P,DJ,0.268622,0.681761,0.592681,0.354893,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,A,D,B,D,E,A,A,P,A,A,A,A,F,E,G,E,B,AB,EG,A,E,I,C,J,HA,0.553846,0.299102,0.263570,0.696873,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [830]:
pandas_profiling.ProfileReport(train)

Number of variables,132
Number of observations,188318
Total Missing (%),0.0%
Total size in memory,189.7 MiB
Average record size in memory,1.0 KiB
Numeric,14
Categorical,116
Boolean,0
Date,0
Text (Unique),0
Rejected,2


In [0]:
train = train.drop(columns=['cont9','cont12'],axis=1)
test = test.drop(columns=['cont9','cont12'],axis=1)

In [882]:
train.shape

(188318, 130)

In [883]:
test.shape

(125546, 129)

In [0]:
ID = train['id']
ID_test = test['id']

In [0]:
train = train.drop(columns='id',axis=1)
test = test.drop('id',axis=1)

In [0]:
A = train['cat109'].value_counts().index[:2]
B = test['cat109'].value_counts().index[:2]

In [0]:
train['cat109'] = np.where(train['cat109'].isin(A), train['cat109'], 'OTHERS_109')
test['cat109'] = np.where(test['cat109'].isin(B), test['cat109'], 'OTHERS_109')

In [888]:
train['cat109'].value_counts()

BI            152918
AB             21933
OTHERS_109     13467
Name: cat109, dtype: int64

In [0]:
C = train['cat110'].value_counts().index[:6]
D = test['cat110'].value_counts().index[:6]

In [0]:
train['cat110'] = np.where(train['cat110'].isin(C), train['cat110'], 'OTHERS_110')
test['cat110'] = np.where(test['cat110'].isin(D), test['cat110'], 'OTHERS_110')

In [891]:
train['cat110'].value_counts()

OTHERS_110    58511
CL            25305
EG            24654
CS            24592
EB            21396
CO            17495
BT            16365
Name: cat110, dtype: int64

In [0]:
E = train['cat112'].value_counts().index[:11]
F = test['cat112'].value_counts().index[:11]

In [0]:
train['cat112'] = np.where(train['cat112'].isin(E), train['cat112'], 'OTHERS_112')
test['cat112'] = np.where(test['cat112'].isin(F), test['cat112'], 'OTHERS_112')

In [894]:
train['cat112'].value_counts()

OTHERS_112    55418
E             25148
AH            18639
AS            17669
J             16222
AF             9368
AN             9138
N              8453
U              8356
AV             7122
AK             6726
K              6059
Name: cat112, dtype: int64

In [0]:
G = train['cat113'].value_counts().index[:11]
H = test['cat113'].value_counts().index[:11]

In [0]:
train['cat113'] = np.where(train['cat113'].isin(G), train['cat113'], 'OTHERS_113')
test['cat113'] = np.where(test['cat113'].isin(H), test['cat113'], 'OTHERS_113')

In [897]:
train['cat113'].value_counts()

OTHERS_113    64830
BM            26191
AE            22030
L             13058
AX            12661
Y             11374
K              7738
S              7033
X              7016
AF             6079
AN             5214
H              5094
Name: cat113, dtype: int64

In [0]:
I = train['cat116'].value_counts().index[:7]
J = test['cat116'].value_counts().index[:7]

In [0]:
train['cat116'] = np.where(train['cat116'].isin(I), train['cat116'], 'OTHERS_116')
test['cat116'] = np.where(test['cat116'].isin(J), test['cat116'], 'OTHERS_116')

In [900]:
train['cat116'].value_counts()

OTHERS_116    106422
HK             21061
DJ             20244
CK             10162
DP              9202
GS              8736
CR              6862
HX              5629
Name: cat116, dtype: int64

In [901]:
train.shape

(188318, 129)

In [902]:
test.shape

(125546, 128)

In [903]:
train.dtypes

cat1       object
cat2       object
cat3       object
cat4       object
cat5       object
           ...   
cont10    float64
cont11    float64
cont13    float64
cont14    float64
loss      float64
Length: 129, dtype: object

In [904]:
test.dtypes

cat1       object
cat2       object
cat3       object
cat4       object
cat5       object
           ...   
cont8     float64
cont10    float64
cont11    float64
cont13    float64
cont14    float64
Length: 128, dtype: object

In [0]:
cat_cols = train.select_dtypes("object").columns
num_cols = train.columns.difference(cat_cols)

In [906]:
cat_cols

Index(['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9',
       'cat10',
       ...
       'cat107', 'cat108', 'cat109', 'cat110', 'cat111', 'cat112', 'cat113',
       'cat114', 'cat115', 'cat116'],
      dtype='object', length=116)

In [907]:
num_cols

Index(['cont1', 'cont10', 'cont11', 'cont13', 'cont14', 'cont2', 'cont3',
       'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'loss'],
      dtype='object')

In [0]:
cat_cols_test = test.select_dtypes("object").columns
num_cols_test = test.columns.difference(cat_cols_test)

In [909]:
cat_cols_test

Index(['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9',
       'cat10',
       ...
       'cat107', 'cat108', 'cat109', 'cat110', 'cat111', 'cat112', 'cat113',
       'cat114', 'cat115', 'cat116'],
      dtype='object', length=116)

In [910]:
num_cols_test

Index(['cont1', 'cont10', 'cont11', 'cont13', 'cont14', 'cont2', 'cont3',
       'cont4', 'cont5', 'cont6', 'cont7', 'cont8'],
      dtype='object')

In [0]:
all_Data = []

In [0]:
train_1 = train.copy()

In [0]:
train = train.drop('loss',axis=1)

In [0]:
train_dummies = pd.get_dummies(train,columns= cat_cols,drop_first=True)

In [915]:
train_dummies.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont10,cont11,cont13,cont14,cat1_B,cat2_B,cat3_B,cat4_B,cat5_B,cat6_B,cat7_B,cat8_B,cat9_B,cat10_B,cat11_B,cat12_B,cat13_B,cat14_B,cat15_B,cat16_B,cat17_B,cat18_B,cat19_B,cat20_B,cat21_B,cat22_B,cat23_B,cat24_B,cat25_B,cat26_B,cat27_B,cat28_B,...,cat114_J,cat114_L,cat114_N,cat114_O,cat114_Q,cat114_R,cat114_S,cat114_U,cat114_V,cat114_W,cat114_X,cat115_B,cat115_C,cat115_D,cat115_E,cat115_F,cat115_G,cat115_H,cat115_I,cat115_J,cat115_K,cat115_L,cat115_M,cat115_N,cat115_O,cat115_P,cat115_Q,cat115_R,cat115_S,cat115_T,cat115_U,cat115_W,cat115_X,cat116_CR,cat116_DJ,cat116_DP,cat116_GS,cat116_HK,cat116_HX,cat116_OTHERS_116
0,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.83510,0.569745,0.822493,0.714843,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.43919,0.338312,0.611431,0.304496,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.32446,0.381398,0.195709,0.774425,0,1,0,0,1,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.44467,0.327915,0.605077,0.602642,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.21230,0.204687,0.246011,0.432606,0,1,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
all_Data = train_dummies

In [917]:
all_Data.shape

(188318, 424)

In [0]:
test = pd.get_dummies(test,columns= cat_cols_test,drop_first=True)

In [919]:
test.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont10,cont11,cont13,cont14,cat1_B,cat2_B,cat3_B,cat4_B,cat5_B,cat6_B,cat7_B,cat8_B,cat9_B,cat10_B,cat11_B,cat12_B,cat13_B,cat14_B,cat15_B,cat16_B,cat17_B,cat18_B,cat19_B,cat20_B,cat21_B,cat22_B,cat23_B,cat24_B,cat25_B,cat26_B,cat27_B,cat28_B,...,cat114_I,cat114_J,cat114_L,cat114_N,cat114_O,cat114_Q,cat114_R,cat114_S,cat114_U,cat114_V,cat114_W,cat115_B,cat115_C,cat115_D,cat115_E,cat115_F,cat115_G,cat115_H,cat115_I,cat115_J,cat115_K,cat115_L,cat115_M,cat115_N,cat115_O,cat115_P,cat115_Q,cat115_R,cat115_S,cat115_T,cat115_U,cat115_W,cat115_X,cat116_CR,cat116_DJ,cat116_DP,cat116_GS,cat116_HK,cat116_HX,cat116_OTHERS_116
0,0.321594,0.299102,0.246911,0.402922,0.281143,0.466591,0.317681,0.61229,0.38016,0.377724,0.704052,0.392562,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.634734,0.620805,0.654310,0.946616,0.836443,0.482425,0.443760,0.71330,0.60401,0.689039,0.453468,0.208045,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0.290813,0.737068,0.711159,0.412789,0.718531,0.212308,0.325779,0.29758,0.30529,0.245410,0.258586,0.297232,0,1,0,1,1,0,1,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.268622,0.681761,0.592681,0.354893,0.397069,0.369930,0.342355,0.40028,0.31480,0.348867,0.592264,0.555955,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0.553846,0.299102,0.263570,0.696873,0.302678,0.398862,0.391833,0.23688,0.50556,0.359572,0.301535,0.825823,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
cols_to_drop_train = set(all_Data.columns.tolist()) -set(test.columns.tolist())

In [0]:
cols_to_drop_test = set(test.columns.tolist()) -set(all_Data.columns.tolist()) 

In [0]:
all_Data = all_Data.drop(cols_to_drop_train,axis=1)

In [0]:
test = test.drop(cols_to_drop_test,axis=1)

In [924]:
all_Data.shape

(188318, 413)

In [925]:
test.shape

(125546, 413)

In [0]:
X = all_Data

In [0]:
y = train_1['loss']

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

Decision Tree 

In [0]:
regressor = DecisionTreeRegressor(splitter='random',random_state = 0)

In [930]:
regressor.fit(X, y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='random')

In [0]:
y_pred_train = regressor.predict(X_train)
y_pred_test = regressor.predict(X_val)

In [0]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_val, y_pred_test))

In [945]:
print('Decision Tree:')
print('root mean squared error for train: ', rmse_train)
print('root mean squared error validation: ', rmse_test)

Decision Tree:
root mean squared error for train:  0.2039397061609014
root mean squared error validation:  0.3115206837135167


Prediction on Test Data

In [0]:
test_predict_dt = regressor.predict(test)

In [0]:
test_predict_dt = pd.DataFrame(test_predict_dt,index = None,columns=["loss"])

In [0]:
test_submission_id = pd.DataFrame(submission_file,columns=["id"])

In [0]:
full_test_dt = pd.concat([test_submission_id,test_predict_dt],axis=1)

In [940]:
full_test_dt.shape

(125546, 2)

In [0]:
full_test_dt.to_csv("loss_Predictions.csv",index=False,header=True)

In [947]:
print("MAE:", metrics.mean_absolute_error(y_val, y_pred_test))
print('MSE:', metrics.mean_squared_error(y_val, y_pred_test))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_val, y_pred_test)))

MAE: 0.0013106237609742295
MSE: 0.09704513638133692
RMSE: 0.3115206837135167
